<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>
<p>In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.</p>
<p>For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.</p>
<p>Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.</p>

<h2>Step 1 - Scrap Wikipedia for data using BeautifulSoup</h2>
<p>Use the Notebook to build the code to scrape the following Wikipedia page, <a href='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' target='_blank'>https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M</a>, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:</p>
<div> 
    To create the dataframe:
    <ul>
        <li>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.</li>
        <li>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</li>
        <li>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</li>
        <li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.</li>
        <li>Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.</li>
        <li>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.</li>
    </ul>
</div>

<p> Note: There are different website scraping libraries and packages in Python. One of the most common packages is BeautifulSoup. Here is the package's main documentation page: <a href='http://beautiful-soup-4.readthedocs.io/en/latest/' target='_blank'>http://beautiful-soup-4.readthedocs.io/en/latest/</a></p> 

In [1]:
#install Beautiful Soup and requests for Web Scaping
!pip install BeautifulSoup4
!pip install requests

<h3> Step 1A: Get Data </h3>
<ol>
    <li>Get HTML from wikipedia</li>
    <li> Use BeautifySoup to parse html data </li>
    <li> Store parsed data into Pandas DataFrame </li>
</ol>

In [7]:
#imports
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

#get html from wiki page and create soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

#using soup object, iterate the .wikitable to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
canada_df = pd.DataFrame(data = data,columns = columns)
canada_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
canada_df = canada_df.rename(columns = {'Postal Code': 'Postcode'}, inplace = False)
canada_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h3>Step 1B: Data Cleanup</h3>
<ol>
    <li> Remove Boroughs that are 'Not assigned' </li>
    <li> More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma</li>
    <li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough</li>
</ol>

In [9]:
#Remove Boroughs that are 'Not assigned'
canada_df = canada_df[canada_df['Borough'] != 'Not assigned']
canada_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
# More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma
canada_df["Neighbourhood"] = canada_df.groupby("Postcode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))

#remove duplicates
canada_df = canada_df.drop_duplicates()

#update index to be postcode if it isn't already
if(canada_df.index.name != 'Postcode'):
    canada_df = canada_df.set_index('Postcode')
    
canada_df.head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [13]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
mask = canada_df['Neighbourhood'] == "Not assigned"
canada_df.loc[mask, 'Neighbourhood'] = canada_df.loc[mask, 'Borough']

In [14]:
canada_df.head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h3>Step 1C - Output</h3>
<p>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.</p>

In [15]:
canada_df.shape

(103, 2)